# Interface conversacional - Chatbot com LLM AI21

> *Este caderno deve funcionar bem com o kernel **`Data Science 3.0`** no SageMaker Studio*

Neste caderno, criaremos um chatbot usando modelos de base (FM) no Amazon Bedrock. Para nosso caso de uso, utilizamos o Jurassic como FM para criar o chatbot.

## Visão geral

Interfaces conversacionais, como chatbots e assistentes virtuais, podem ser usadas para aprimorar a experiência do usuário para seus clientes. Os chatbots usam processamento de linguagem natural (NLP) e algoritmos de machine learning para entender e responder às perguntas do usuário. Chatbots podem ser utilizados em diversas aplicações, como atendimento ao cliente, vendas e comércio eletrônico, para oferecer respostas rápidas e eficientes aos usuários. É possível acessá-los por vários canais, como sites, plataformas de redes sociais e aplicações de mensagens.


## Chatbot que utiliza o Amazon Bedrock

![Amazon Bedrock - Interface conversacional](./images/chatbot_bedrock.png)


## Casos de uso

1. **Chatbot (básico)** - Chatbot Zero Shot com um modelo FM
2. **Chatbot que utiliza prompt** - modelo (LangChain) - Chatbot com contexto oferecido no modelo do prompt
3. **Chatbot com persona** - Chatbot com funções definidas, por exemplo, coach de carreira e interações humanas
4. **Chatbot com consciência contextual** - Inclusão de contexto por meio de um arquivo externo gerando incorporações.

## Framework LangChain para criação de chatbot com o Amazon Bedrock
Em interfaces conversacionais, como chatbots, é extremamente importante se lembrar de interações anteriores, seja em curto ou longo prazo.

O LangChain oferece componentes de memória de duas formas. Primeiro, o LangChain oferece recursos auxiliares para gerenciar e manipular mensagens de chat anteriores. Eles são projetados para serem modulares e úteis independentemente de como são usados. Em segundo lugar, o LangChain oferece maneiras fáceis de incorporar esses recursos nas cadeias.
Ele nos permite definir e interagir com diferentes tipos de abstrações, o que facilita a criação de chatbots poderosos.

## Criação de chatbot com contexto - Principais elementos

O primeiro processo na criação de um chatbot com consciência contextual é **gerar incorporações** para o contexto. Normalmente, existe um processo de ingestão que será executado no modelo de incorporação e gera as incorporações que serão armazenadas em um tipo de armazenamento de vetores. Neste exemplo, usaremos o Titan Embeddings para fazer isso.

![Embeddings](./images/embeddings_lang.png)

O segundo processo é a orquestração da solicitação do usuário, interação, invocação e entrega dos resultados.

![Chatbot](./images/chatbot_lang.png)

## Arquitetura [Chatbot com consciência de contexto]
![4](./images/context-aware-chatbot.png)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Chatbot (básico - sem contexto)

#### Como usar o CoversationChain do LangChain para iniciar uma conversa
Chatbots precisam se lembrar das interações anteriores. Isso é possível graças à memória conversacional. Podemos implementar a memória conversacional de diversas formas. No contexto do LangChain, todas são criadas baseadas no ConversationChain.

Observação: as saídas do modelo não são deterministas

In [ ]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory

ai21_llm = Bedrock(model_id="ai21.j2-ultra-v1", client=boto3_bedrock)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=ai21_llm, verbose=True, memory=memory
)

try:
    
    print_ww(conversation.predict(input="Hi there!"))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

#### Novas perguntas

O modelo respondeu com uma mensagem inicial, vamos fazer algumas perguntas

In [ ]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

#### Aprimorar as perguntas

Vamos fazer uma pergunta sem mencionar a palavra “jardim” para ver se o modelo consegue entender a conversar anterior

In [ ]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

#### Encerrar a conversa

In [ ]:
print_ww(conversation.predict(input="That's all, thank you!"))

## Chatbot usando modelo de prompt (Langchain)

O PromptTemplate é responsável pela construção desta entrada. O LangChain oferece várias classes e funções para facilitar a criação e trabalho com prompts. Usaremos o modelo de prompt padrão aqui. [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

chat_history = []

# turn verbose to true to see the full logs and documents
qa= ConversationChain(
    llm=ai21_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

print(f"ChatBot:DEFAULT:PROMPT:TEMPLATE: is ={qa.prompt.template}")

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Vamos começar uma conversa

In [ ]:
chat = ChatUX(qa)
chat.start_chat()

## Chatbot com persona

O assistente de IA desempenhará a função de coach de carreira. O diálogo de dramatização exige que a mensagem do usuário seja definida antes de iniciar a conversa. O ConversationBufferMemory é usado para popular antecipadamente o diálogo

In [ ]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Context:You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am career coach and give career advice")
ai21_llm = Bedrock(model_id="ai21.j2-ultra-v1",client=boto3_bedrock)
conversation = ConversationChain(
     llm=ai21_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="What are the career options in AI?"))

##### Vamos perguntar algo que não seja a especialidade desta Persona e o modelo não deve responder e explicar o motivo

In [ ]:
conversation.verbose = False
print_ww(conversation.predict(input="How to fix my car?"))

## Chatbot com contexto 
Neste caso de uso, vamos pedir que o chatbot responda a uma pergunta relacionada ao contexto recebido. Utilizaremos um arquivo csv e o modelo de incorporações do Titan para criar o vetor. Esse vetor será armazenado no FAISS. Quando o chatbot receber uma pergunta, passaremos esse vetor e recuperaremos a resposta. 

#### Modelo de incorporações do Titan

As incorporações são uma forma de representar palavras, frases e qualquer outro item discreto como vetores em um espaço de vetores contínuo. Isso permite que modelos de machine learning realizem operações matemáticas nessas representações e capturem relações semânticas entre elas.


Isso será usado para o RAG [capacidade de pesquisar documentos](https://labelbox.com/blog/how-vector-similarity-search-works/) 


In [ ]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

#### FAISS como VectorStore

Para poder usar as incorporações para pesquisa, precisamos de um armazenamento que possa realizar pesquisas de similaridades de vetores de modo eficiente. Neste caderno, usaremos o FAISS, que é um armazenamento de memória. Para armazenar vetores permanentemente, você pode usar o pgVector, Pinecone, Weaviate ou Chroma.

As APIs VectorStore do LangChain estão disponíveis [aqui](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

Para saber mais sobre o armazenamento de vetores FAISS, consulte este [documento](https://arxiv.org/pdf/1702.08734.pdf).

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

s3_path = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

#### Para executar um teste rápido de baixo código 

Podemos usar uma classe do Wrapper fornecida pelo LangChain para consultar o armazenamento da base de dados de vetores e receber os documentos relevantes. Nos bastidores, isso apenas executará uma cadeia de QA com todos os valores padrão

In [ ]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=ai21_llm))

#### Aplicação do Chatbot

Para o chatbot, precisamos de gerenciamento de contexto, histórico, armazenamentos de vetores e muito mais. Começaremos com um ConversationalRetrievalChain

Ele usa memória conversacional e RetrievalQAChain, o que permite incluir históricos da conversa que podem ser usados para as próximas perguntas. Fonte: https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

Defina verbose como True para ver tudo que acontece nos bastidores

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT


def create_prompt_template():
    _template = """{chat_history}

Answer only with the new question.
How would you ask the question considering the previous conversation: {question}
Question:"""
    CONVO_QUESTION_PROMPT = PromptTemplate.from_template(_template)
    return CONVO_QUESTION_PROMPT

memory_chain = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
chat_history=[]

#### Parâmetros usados para ConversationRetrievalChain
* **retriever**: usamos `VectorStoreRetriever`, que é baseado em um `VectorStore`. Você pode escolher dois tipos de pesquisa para recuperar texto: `"similarity"` ou `"mmr"`. `search_type="similarity"` usa a pesquisa por similaridade no objeto de recuperação, no qual seleciona vetores de blocos de texto mais semelhantes ao vetor da pergunta.

* **memory**: cadeia de memória para armazenar o histórico 

* **condense_question_prompt**: ao receber uma pergunta do usuário, usamos a conversa anterior e essa pergunta para criar uma pergunta única

* **chain_type**: se o histórico da conversa for longo e não se encaixar no contexto, use este parâmetro e as opções são `stuff`, `refine`, `map_reduce`, `map-rerank`

Se a pergunta feita estiver fora do escopo do contexto, o modelo dirá que não sabe a resposta

**Observação**: se tiver curiosidade para saber como a cadeia funciona, retire o comentário da linha `verbose=True`.

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=ai21_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    #verbose=True,
    #condense_question_prompt=CONDENSE_QUESTION_PROMPT, # create_prompt_template(), 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=100
)

qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}:

Use at maximum 3 sentences to answer the question. 

{question}:

If the answer is not in the context say "Sorry, I don't know, as the answer was not found in the context."

Answer:""")

Vamos começar uma conversa

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

### Nesta demonstração, usamos o LLM Titan para criar uma interface conversacional com os seguintes padrões:

1. Chatbot (básico - sem contexto)

2. Chatbot usando modelo de prompt (LangChain)

3. Chatbot com personas

4. Chatbot com contexto